In [73]:
import pymysql
import pandas as pd
import boto3

In [124]:
def database_connect():
    conn = pymysql.connect(host="hercules-db.cwm9hg1irjsb.ap-northeast-1.rds.amazonaws.com"\
                           , user = "hercules", password="*********",db='src_db')
    cursor=conn.cursor()

    cursor.execute(create_query)
    cursor.execute(create_query2)

    



In [127]:
def populate_table(object_list):
    obj_list_filter = object_list[:3]
    for obj in obj_list_filter:
        file = s3.get_object(Bucket=bucket,Key=obj['Key'])
        obj_df = pd.read_csv(file["Body"],delimiter=",")
        year = year_dict[obj["Key"][-6:-4]]
        obj_df["year"]=year
        obj_df.fillna("",inplace=True)
        df_list = obj_df.values.tolist()
        cursor.executemany(insert_query,df_list) 

In [128]:
def helper_func(object_list):
    obj_list_filter = object_list[3:]
    for obj in obj_list_filter:
        file = s3.get_object(Bucket=bucket,Key=obj['Key'])
        obj_df = pd.read_csv(file["Body"],delimiter=",")
        year = year_dict[obj["Key"][-6:-4]]
        obj_df["year"]=year
        obj_df.fillna("",inplace=True)
        df_list = obj_df.values.tolist()
        cursor.executemany(insert_query2,df_list)
    conn.commit()  

In [129]:
def s3_to_rds():
    s3 = boto3.client("s3",region_name='ap-northeast-1', 
                      aws_access_key_id="*************", 
                      aws_secret_access_key="V*************************")
    response = s3.list_objects(Bucket=bucket, Prefix="raw_files/export_")
    files = response["Contents"]
    populate_table(files)
    helper_func(files)

In [130]:
bucket = "import-export-analysis-raw"
year_dict = {"01":"2015","02":"2016","03":"2017","04":"2018","05":"2019","06":"2020"}
insert_query = "insert into export_all_years (product_code,product,unit,country_code\
,country_name, quantity,value,year) values (%s,%s,%s,%s,%s,%s,%s,%s)"
create_query = '''create table export_all_years ( id int not null auto_increment,product_code varchar(255),product varchar(255),unit varchar(255),country_code varchar(255)\
,country_name varchar(255), quantity varchar(255),value varchar(255),year varchar(255), primary key (id) )'''
create_query2 = '''
create table export_raw ( id int not null auto_increment,product varchar(255)\
,country_name varchar(255), unit varchar(255), quantity varchar(255),value varchar(255),year varchar(255), primary key (id) )'''
insert_query2 = "insert into export_raw (product,\
country_name,unit, quantity,value,year) values (%s,%s,%s,%s,%s,%s)"
database_connect()
s3_to_rds()

In [141]:
cursor.execute("select * from export_all_years where year='2017'")
cursor.fetchall()

((41602,
  '99',
  'OTHER COMMODITIES',
  '',
  '599',
  'UNSPECIFIED',
  '',
  '270.889542',
  '2017'),
 (41603,
  '99',
  'OTHER COMMODITIES',
  '',
  '1213',
  'KENYA',
  '',
  '17.159452',
  '2017'),
 (41604,
  '99',
  'OTHER COMMODITIES',
  '',
  '1395',
  'TANZANIA REP',
  '',
  '8.896539',
  '2017'),
 (41605,
  '99',
  'OTHER COMMODITIES',
  '',
  '1417',
  'UGANDA',
  '',
  '5.181412',
  '2017'),
 (41606,
  '99',
  'OTHER COMMODITIES',
  '',
  '2035',
  'BENIN',
  '',
  '2.476015',
  '2017'),
 (41607,
  '99',
  'OTHER COMMODITIES',
  '',
  '2050',
  'BURKINA FASO',
  '',
  '0.452',
  '2017'),
 (41608,
  '99',
  'OTHER COMMODITIES',
  '',
  '2057',
  'CAMEROON',
  '',
  '0.448168',
  '2017'),
 (41609,
  '99',
  'OTHER COMMODITIES',
  '',
  '2067',
  'C AFRI REP',
  '',
  '0.001924',
  '2017'),
 (41610,
  '99',
  'OTHER COMMODITIES',
  '',
  '2069',
  'CHAD',
  '',
  '0.004635',
  '2017'),
 (41611,
  '99',
  'OTHER COMMODITIES',
  '',
  '2087',
  'CONGO P REP',
  '',
  '1.747451'